In [4]:
import pandas as pd
import numpy as np
from utils import retrieve_data

In [2]:
# Retrieve Data
data = retrieve_data()
train = data['train']
test = data['test']
train_num = data['train_num']
y_feature = 'SalePrice'

# Implementing mini-batch gradient descent:
        1. Having a number for batch-size
        2. Going through the whole dataset in each batch
        3. passing the parameters along from each batch to the other one set_params(**args) /get_params
        
--------------------------------------------------------------
Making a mock dataset with non-missing values then run it on that and test

In [22]:
# Making a mock slice of the data
num_feats = list(data['train_num'].columns)[1:-1]
data['train_num_missing']
num_feats.remove(data['train_num_missing'][0])
num_feats.remove(data['train_num_missing'][1])
num_feats.remove(data['train_num_missing'][2])
mock_feats = train.corr()['SalePrice'][num_feats].nlargest(7).keys().to_list()
mock_feats.append('SalePrice')
mock_data = train[mock_feats]
mock_feats.remove('SalePrice')

In [23]:
mock_feats

['OverallQual',
 'GrLivArea',
 'GarageCars',
 'GarageArea',
 'TotalBsmtSF',
 '1stFlrSF',
 'FullBath']

In [16]:
mock_data.shape

(1460, 8)

In [17]:
def split_data(data, batch_size):
    """
        Breaks the data into batches so it could be used in the mini-batch
        gradient descent.
        
        # Arguments:
            data: dataframe containing all the data
            batch_size: the sizes batches
        
        # Returns:
            a list containing a slices of dataset
    """
    num = data.shape[0] # Number of data points
    batch_num = num // batch_size + 1 # Number of batches
    batches = [] # List containig the slices of data
    
    for i in range(batch_num):
        print(i)
        print(data[i * batch_size:(i + 1) * batch_size].shape)
        # Implement: given that some values will go over the size of
        # the data write a function to randomly select the remianing
        # of the batch
        batches.append(data[i * batch_size:(i + 1) * batch_size])
        
    return batches

In [18]:
s = split_data(mock_data, batch_size=500)

0
(500, 8)
1
(500, 8)
2
(460, 8)


In [24]:
s[0][mock_feats]

,OverallQual,GrLivArea,GarageCars,GarageArea,TotalBsmtSF,1stFlrSF,FullBath
0,7,1710,2,548,856,856,2
1,6,1262,2,460,1262,1262,2
2,7,1786,2,608,920,920,2
3,7,1717,3,642,756,961,1
4,8,2198,3,836,1145,1145,2
...,...,...,...,...,...,...,...
495,4,720,0,0,720,720,1
496,8,3228,2,546,3200,3228,3
497,7,1820,1,240,1026,1133,2
498,5,899,1,288,864,899,1


In [64]:
# GradientBoostingRegressor

-5.459258327919444e+22
1.5354429531650208e+16
